In [9]:
from owlready2 import *
import shutil

In [10]:
from util import file_util
from util.graph_util import Graph
from service import python_content_creator

In [18]:
member_propagation = {}
cp_map = {}
template = """
{import_statements}
class {class_name}({parent}):
    {functions}
"""
def generate_imports_from_template(node):
    return None

def generate_class_from_template(node, parent, functions, imports):
    return template.format(
        import_statements=imports, 
        class_name=node.label.first(), 
        parent= parent.label.first() if parent else "", 
        functions=functions)

def function_name_handler(func_name):
    if func_name == "init":
        return "__init__"
    return func_name

        
        
def generate_function_body_from_template(node, func, target):
    print("STM: ", func.label.first())
    if func.label.first() == "init":
        stmts = ["self.{var} = {var}".format(var=obj.label.first()) for obj in target]
        stmt = "\n\t".join(stmts)
        params = get_inherited_params(node)
        if params:
            stmt = stmt + "\n\t{parent}.__init__(self, {params})".format(
                parent=cp_map[node].label.first(), params=params)
        return stmt

def generate_function_param_from_template(node, target):
    params = ""
    param_template = "{var} = {value}"
    for obj in target:
        if not obj.default:
            param = obj.label.first()
        else:
            param = param_template.format(
                var=obj.label.first(), value=obj.default.first())
        params = params + ", " + param
    inherited_params = get_inherited_params(node)
    if inherited_params:
        params = ", " + inherited_params + params 
    return params

def generate_function_from_template(node, func):
    func_template = """
    def {func_name}(self{params}):
        {statements}
    
    """
    print("LABEL: ",func.label[0])
    var = func.label.first()
    target = eval("node."+var)
    print("TARGET: ",target)
    if var == "init":
        for child in node.descendants():
            if child != node:
                if member_propagation.get(child):
                    member_propagation[child].extend(target)
                else:
                    member_propagation[child] = target
    if target:
        func = func_template.format(
            func_name=function_name_handler(func.label.first()),
            params=generate_function_param_from_template(node, target),
            statements=generate_function_body_from_template(node, func, target)
        )
        #print(func)
        #print(target[0].label.first())
        return func

def get_inherited_params(node):
    inherited_variables = member_propagation.get(node)
    if inherited_variables:
        var = [obj.label.first() for obj in inherited_variables]
        params = ", ".join(var)
        return params
    return ""
    
def generate_init_by_member_propagation(node):
    func_template = """
    def __init__(self, {params}):
        {parent}.__init__(self, {params})
    """
    return func_template.format(
        params=get_inherited_params(node),
        parent=cp_map[node].label.first()
    )
    
    
def generate_functions_from_template(node):
    function_names = set()
    func_data = ""
    for func in node.get_class_properties():
        if func.label:
            func_data = func_data + generate_function_from_template(node, func)
            function_names.add(func.label[0])
    #print(func_data)
    if not func_data and member_propagation.get(node):
            func_data = generate_init_by_member_propagation(node)
    return func_data
        

def create_file_contents(file, node, child_parent_map):
    print(node.label)
    global cp_map
    cp_map = child_parent_map
    parent = cp_map[node]
    if parent:
        print("PARENT: ", parent)
    func_data = generate_functions_from_template(node)
    if not func_data:
        func_data = "pass"
    print(generate_class_from_template(node, parent, func_data, None))
    #print("MEM_PROP: ", member_propagation)

In [19]:
child_parent_map = {}
onto = get_ontology("ml_algorithms.owl").load()
import os
queue = []
root_class = None
#print(onto.classes())
print([i for i in onto.annotation_properties()])
for onto_class in list(onto.classes()):
    #print(onto_class.label)
    if(onto_class.label[0] == 'MLalgorithms'):
        #print(onto_class.label)
        root_class = onto_class

queue.append(root_class)
dir_structure = [root_class.label[0]]
file_path = []
child_parent_map[root_class] = None
while queue:
    node = queue.pop(0)
    #print("Node: ", node, "DESC: ", node.descendants())
    file = dir_structure.pop(0)
    if onto.get_children_of(node):
        file_util.create_folders_and_subfolders(file)
    file_util.create_file(file + ".py")
    create_file_contents(file, node, child_parent_map)
    file_path.append(file)
    for child in onto.get_children_of(node):
        queue.append(child)
        child_parent_map[child] = node
        dir_structure.append(os.path.join(file, child.label[0]))

del member_propagation
del cp_map
print(file_path)

[ml_algorithms.default, ml_algorithms.mla:default, ml_algorithms.mla:dtype]
['MLalgorithms']

None
class MLalgorithms():
    pass

['Bayesian']
PARENT:  ml_algorithms.MLalgoritthms

None
class Bayesian(MLalgorithms):
    pass

['Classification']
PARENT:  ml_algorithms.MLalgoritthms

None
class Classification(MLalgorithms):
    pass

['Clustering']
PARENT:  ml_algorithms.MLalgoritthms

None
class Clustering(MLalgorithms):
    pass

['Regression']
PARENT:  ml_algorithms.MLalgoritthms
LABEL:  init
TARGET:  [ml_algorithms.normalize, ml_algorithms.copy_X, ml_algorithms.fit_intercept, ml_algorithms.n_jobs, ml_algorithms.max_iter]
STM:  init
LABEL:  predict
TARGET:  [ml_algorithms.X]
STM:  predict
LABEL:  fit
TARGET:  [ml_algorithms.sample_weight, ml_algorithms.y, ml_algorithms.X]
STM:  fit

None
class Regression(MLalgorithms):
    
    def __init__(self, normalize, copy_X, fit_intercept, n_jobs = None, max_iter):
        self.normalize = normalize
	self.copy_X = copy_X
	self.fit_intercept = 

In [ ]:
for i in list(root_class.descendants()):
    if i.label[0] == 'Regression':
        for j in list(i.descendants()):
            print(j.label[0])
            for k in list(onto.object_properties()):
                try:
                    print("df",k)
                except:
                    pass

In [72]:
class ML:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def fit(self, X, y):
        self._model.printf(X)

In [73]:
class student:
    def printf(self, X):
        print(X)

In [76]:
class DT(ML):
    def __init__(self, d, e):
        self.x = d
        self.y = e
        ML.__init__(self, d,e)
        self._model = student()
    
    def printl(self):
        print(self.b)

In [77]:
tree = DT(5,4)
tree.fit(5,4)
tree.printl()
tree._model.printf(5)

5
4
5


In [18]:
class A:
    def __init__(self, name):
        self.name = name
        print("A")

class B(A):
    def __init__(self, name):
        print("B")
        A.__init__(self, name=name)

In [16]:
class C(B):
    def __init_(self, a, b):
        self.b = b
        print("C")
        B.__init__(self, a)

In [19]:
c = B("varun")

B
A


In [20]:
c.name

'varun'

In [52]:
target = [1,2,3,4]

In [53]:
d = {}

In [54]:
d["A"] = [1]

In [55]:
d["A"].extend(target)

In [41]:
d["A"] = d["A"].append(target)

In [44]:
d["A"].append(target)

In [71]:
a =  True
b = False

In [79]:
for i in None:
    print("HI")

TypeError: 'NoneType' object is not iterable